In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.feature_selection import SelectKBest, SelectFromModel, RFE, RFECV, SelectPercentile, SelectFpr, SelectFdr, SelectFwe
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, SelectFdr
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import matplotlib.pyplot as plt
import pickle
from sksurv.metrics import concordance_index_censored
import lifelines as ll
# from lifelines.utils.sklearn_adapter import sklearn_adapter
# CoxRegression = sklearn_adapter(ll.CoxPHFitter, event_col = 'event')
import sys
sys.path.append('/odinn/users/thjodbjorge/Python_functions/')
import Predict_functions as pf
from Calculate_score import calculate_metrics, make_class_table
from R_functions import R_pROC,R_pROC_compareROC,R_pROC_compareROC_boot, R_pROC_AUC, R_timeROC, R_timeROC_CI, R_timeROC_pval, R_NRIbin,R_NRIcens,R_NRIcensipw, R_censROC, R_hoslem, R_Greenwood_Nam


# raw_data = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/raw_with_info.csv',index_col = 'Barcode2d' )
probe_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probe_info.csv', index_col = 'SeqId')

pn_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/pn_info_Mor/pn_info_Mor_event.csv',index_col = 'Barcode2d' )
probes_to_skip = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probes_to_skip.txt')['probe']

In [ ]:
folder = '/odinn/users/thjodbjorge/Proteomics/Mortality2/'
feat_folder = 'Features2/'
pred_folder = 'Predictions3/'
plots = 'Plots3/'
save_plot = True

endpoints = ['death']
# endpoints = ['Neoplasm','Nervous','Circulatory','Respiratory','Other','death'] 
time_to_event = pn_info.time_to_death
no_event_before = pn_info.no_death_before
for endpoint in endpoints:
    if endpoint == 'death':
        use_event = pn_info.event_death
    elif endpoint == 'Neoplasm':
        use_event = pn_info.event_death & (pn_info.Cause_of_death == 'Neoplasm')
    elif endpoint == 'Nervous':
        use_event = pn_info.event_death & (pn_info.Cause_of_death == 'Nervous')
    elif endpoint == 'Circulatory':
        use_event = pn_info.event_death & (pn_info.Cause_of_death == 'Circulatory')
    elif endpoint == 'Respiratory':
        use_event = pn_info.event_death & (pn_info.Cause_of_death == 'Respiratory')
    elif endpoint == 'Other':
        use_event = pn_info.event_death & (pn_info.Cause_of_death == 'Other')

y = []
for i in range(1,19):
    y.append(use_event & (time_to_event <= i))

kf = KFold(n_splits=10, random_state=10, shuffle=False) 
I_train_main, I_test_main = train_test_split(pn_info.index, train_size=0.7, random_state = 10)
# I_val_main, I_test_main = train_test_split(I_test_main, train_size=0.5, random_state = 10)


file = open(folder+"{}_keep_samples.pkl".format('Mor'),'rb')
keep_samples_dict = pickle.load(file)

print(keep_samples_dict.keys())

In [ ]:
dataset = 'Old_60105'
HERA_dataset = 'HERA_60105'
all_dataset = 'All_60105'
new_dataset = 'New_60105'

# age =( pn_info.Age_at_sample_collection_2 >=70 )& (pn_info.Age_at_sample_collection_2 < 80) 
# age_ind = pn_info[age].index
try: 
    file = open(folder+pred_folder + "{}_{}_all_prediction.pkl".format(endpoint,dataset),'rb')
    pred_test_dict = pickle.load(file)
except:
    print('No test predictions')
  
# print(pred_dict.keys())

k_plot=1
k = k_plot

plot_folder = '{}_{}/'.format(endpoint,dataset)

keep_samples = keep_samples_dict[dataset]

I_train = I_train_main.intersection(keep_samples)#.intersection(have_prs)
I_test = I_test_main.intersection(keep_samples)#.intersection(have_prs)

hera_samples = keep_samples_dict[HERA_dataset]
old_samples = keep_samples_dict[dataset]
all_samples = keep_samples_dict[all_dataset]
new_samples = keep_samples_dict[new_dataset]

I_old = old_samples
I_use =  hera_samples

y_train = y[k][I_train]
y_test= y[k][I_test]

y_use = y[k][I_use]

In [ ]:
%%capture cap --no-stderr
k = k_plot

keys = ['{}_y{}_agesex_lr'.format(dataset,k),'{}_y{}_tradcancer_lr'.format(dataset,k),
       '{}_y{}_agesexGDF15_lr'.format(dataset,k),'{}_y{}_agesexprotein_l1'.format(dataset,k)]
name_keys = ['Age+sex','Baseline','Age+sex+GDF15','Age+sex+Protein']
fig = plt.figure(figsize = [20,12])
for j,key in enumerate(keys):
    print(keys[j])
    # key = 'predy{}_{}_tradstatproteinprs_coxelnet'.format(k,dataset)
    pred = pred_test_dict[key]    
    pred= pd.DataFrame(pred,index=pn_info.index)
    pred = pred.loc[I_use]

    risk_bins =  np.digitize(pred,np.quantile(pred,[0,0.2,0.8,0.95,1]))


    fig.add_subplot(2,2,j+1)
    KMFs = []
    for i in range(4,0,-1):
        kmf =  ll.fitters.kaplan_meier_fitter.KaplanMeierFitter()
        ind = I_use[risk_bins[:,0]==i]
        kmf.fit(time_to_event[ind],use_event[ind])
        KMFs.append(kmf)
        kmf.plot(loc=slice(0,5))
#         print(len(ind), np.mean(pred.loc[ind]))
        print(kmf.event_table.loc[0,'at_risk'],1- kmf.predict(1),1-kmf.predict(2))
    plt.legend(['95%-100%','80%-95%','20%-80%','0%-20%'])  
    # plt.legend(['0%-5%','5%-20%','20%-50%','50%-80%','80%-95%','95%-100%'])
    plt.axis([0,3,0.7,1.05])
    plt.title(name_keys[j])
    plt.ylabel('Survival')
    plt.xlabel('Time in years')
    plt.grid(True)
#     plt.show()
if save_plot: 
    plt.savefig(folder+plots+plot_folder+'{}_{}_KaplanMeier_5p_y{}_HERA.png'.format(endpoint,dataset,k))


In [ ]:
with open(folder+plots+plot_folder+'{}_{}_KaplanMeier_numbers_5p_y{}_HERA.txt'.format(endpoint,dataset,k), 'w') as f:
    f.write(cap.stdout)
cap.show()
del cap

In [ ]:
I_use.shape

In [ ]:
use_event[I_use].sum()

In [ ]:
use_event[hera_samples].sum()

In [ ]:
pd.set_option('display.max_rows', 100)
pn_info.loc[I_use][use_event[I_use]][['Age_at_sample_collection_2','Time_of_plasma_collection_2','prj','site','ICD-10','no_CAD_before','no_MI_before','cancer','Cause_of_death','Smoker']]

In [ ]:
pn_info.loc[I_use][use_event[I_use]][['Age_at_sample_collection_2','Time_of_plasma_collection_2','prj','site','ICD-10','no_CAD_before','no_MI_before','cancer','Cause_of_death','Smoker']].groupby('Cause_of_death').count()

In [ ]:
pn_info.loc[hera_samples][use_event[hera_samples]][['Age_at_sample_collection_2','Time_of_plasma_collection_2','prj','site','ICD-10','no_CAD_before','no_MI_before','cancer','Cause_of_death','Smoker']].groupby('Cause_of_death').count()

In [ ]:
pn_info[pn_info.prj == '45_LUNG'].groupby('ICD-10').count()

In [ ]:
pn_info[pn_info.prj == '45_LUNG'].shape

In [ ]:
pn_info.loc[I_use][use_event[I_use]][['Age_at_sample_collection_2','Time_of_plasma_collection_2','prj','site','ICD-10','no_CAD_before','no_MI_before','cancer','Cause_of_death','Smoker']].groupby(['prj','Cause_of_death']).count()

In [ ]:
pn_info[pn_info.prj == '62_INFEC'].shape

In [ ]:
pn_info[pn_info.prj == '25_ASTM'].shape

In [ ]:
pn_info.loc[I_use][use_event[I_use]][['Age_at_sample_collection_2','Time_of_plasma_collection_2','prj','site','ICD-10','no_CAD_before','no_MI_before','cancer','Cause_of_death','Smoker']].groupby(['Cause_of_death','prj']).count()

### Simple proteomics model prediction results

In [ ]:
VERY_SMALL_SIZE = 12
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=VERY_SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) 

In [ ]:
dataset = 'Old_18105'
pred = pd.read_csv(folder+pred_folder + "{}_{}_protein_prediction_all.csv".format(endpoint,dataset),index_col = 'Barcode2d')
pred_as = pd.read_csv(folder+pred_folder + "{}_{}_agesex_prediction_all.csv".format(endpoint,dataset),index_col = 'Barcode2d')
pred_baseline = pd.read_csv(folder+pred_folder + "{}_{}_baseline2_prediction_all.csv".format(endpoint,dataset),index_col = 'Barcode2d')

In [ ]:
dataset = 'New_18105'
keep_samples = keep_samples_dict[dataset]

In [ ]:
HERA_death = pn_info.loc[keep_samples][pn_info.loc[keep_samples,'event_death']].index
HERA_living = pn_info.loc[keep_samples][~pn_info.loc[keep_samples,'event_death']].index

In [ ]:
pred_year = 'pred_y1'

print('Age + sex dead people: ', pred_as.loc[HERA_death][pred_year].mean())
print('Baseline dead people: ', pred_baseline.loc[HERA_death][pred_year].mean())
print('Protein dead people: ', pred.loc[HERA_death][pred_year].mean())
print('Age+sex living people: ', pred_as.loc[HERA_living][pred_year].mean())
print('Baseline living people: ', pred_baseline.loc[HERA_living][pred_year].mean())
print('Protein living people: ', pred.loc[HERA_living][pred_year].mean())

In [ ]:

boxprops_pro = dict(color='C3', linewidth=2)    
boxprops_as = dict(color='C0', linewidth=2)  
# boxprops_gdf = dict(color='C2', linewidth=2)   
boxprops_baseline = dict(color='C1', linewidth=2)   

bp1 = plt.boxplot([pred.loc[HERA_death][pred_year],pred.loc[HERA_living][pred_year]], positions = [1,3], boxprops =boxprops_pro)
bp2 = plt.boxplot([pred_baseline.loc[HERA_death][pred_year],pred_baseline.loc[HERA_living][pred_year]], positions = [1.5,3.5], boxprops =boxprops_baseline)
bp3 = plt.boxplot([pred_as.loc[HERA_death][pred_year],pred_as.loc[HERA_living][pred_year]], positions = [2,4], boxprops =boxprops_as)

plt.legend([bp1['boxes'][0],bp2['boxes'][0],bp3['boxes'][0]],['Age+sex+Protein','Baseline','Age+sex'],loc='upper right')
plt.xticks([1.5,3.5],labels = ['Dead','Living'])
plt.ylabel('Predicted risk')
plt.xlabel('')
plt.axis([0,5,-0.05,1])
plt.grid()
plt.savefig(folder+plots+'{}_{}_{}_boxplot_living_vs_dead.png'.format(endpoint,dataset,pred_year))
# plt.show()

In [ ]:
print(roc_auc_score(pn_info.loc[keep_samples,'event_death'],pred_as.loc[keep_samples,pred_year] ))
print(roc_auc_score(pn_info.loc[keep_samples,'event_death'],pred_baseline.loc[keep_samples,pred_year] ))
print(roc_auc_score(pn_info.loc[keep_samples,'event_death'],pred.loc[keep_samples,pred_year] ))

In [ ]:
print(concordance_index_censored(pn_info.loc[keep_samples,'event_death'],pn_info.loc[keep_samples,'time_to_death'],pred_as.loc[keep_samples,pred_year] ))
print(concordance_index_censored(pn_info.loc[keep_samples,'event_death'],pn_info.loc[keep_samples,'time_to_death'],pred_baseline.loc[keep_samples,pred_year] ))
print(concordance_index_censored(pn_info.loc[keep_samples,'event_death'],pn_info.loc[keep_samples,'time_to_death'],pred.loc[keep_samples,pred_year] ))

In [ ]:
pn_info.loc[HERA_death][['Age_at_sample_collection_2','time_to_death']]

In [ ]:
HERA_death.shape

In [ ]:
(pn_info.loc[HERA_death][['time_to_death']]<5).sum()

In [ ]:
# %%capture cap --no-stderr
k = k_plot
pred_year = 'pred_y' + str(k)# k=4
keep_samples = keep_samples[(pn_info.loc[keep_samples,'Age_at_sample_collection_2']>60)&(pn_info.loc[keep_samples,'Age_at_sample_collection_2']<80)]

# pred_as.loc[keep_samples,pred_year] ))
# print(concordance_index_censored(pn_info.loc[keep_samples,'event_death'],pn_info.loc[keep_samples,'time_to_death'],pred_baseline.loc[keep_samples,pred_year] ))
# print(concordance_index_censored(pn_info.loc[keep_samples,'event_death'],pn_info.loc[keep_samples,'time_to_death'],pred.loc[keep_samples,pred_year] 
preds = [[pred_as.loc[keep_samples,pred_year]],[pred_baseline.loc[keep_samples,pred_year]],[pred_pro.loc[keep_samples,pred_year]]]
# keys = ['{}_y{}_agesex_lr'.format(dataset,k),'{}_y{}_baseline2_lr'.format(dataset,k),
#        '{}_y{}_agesexGDF15_lr'.format(dataset,k),'{}_y{}_agesexprotein_l1'.format(dataset,k)]
name_keys = ['Age+sex','Baseline','Age+sex+Protein']
fig = plt.figure(figsize = [10,6])
for j,pred in enumerate(preds):
#     risk_bins =  np.digitize(pred,np.quantile(pred,[0,0.05,0.2,0.8,0.95,1]))
    risk_bins =  np.digitize(pred,np.quantile(pred,[0,0.05,0.95,1]))[0]
#     risk_bins =  np.digitize(pred,np.quantile(pred,[0,0.05,0.95,1]))
#     pred= pd.DataFrame(pred,index=keep_samples)

#     fig.add_subplot(2,2,j+1)
    KMFs = []
    timeline = np.arange(0,5,0.1)
    for i in range(3,0,-1):
        kmf =  ll.fitters.kaplan_meier_fitter.KaplanMeierFitter()
        ind = keep_samples[risk_bins==i]
        kmf.fit(time_to_event[ind],use_event[ind], timeline = timeline)
        KMFs.append(kmf)
        print(kmf.event_table.loc[0,'at_risk'],1- kmf.predict(1),1-kmf.predict(2))
    prob_high_risk = KMFs[0].cumulative_density_at_times(timeline)
    prob_low_risk =  KMFs[-1].cumulative_density_at_times(timeline)
    plt.plot(timeline,(1-prob_high_risk)/(1-prob_low_risk))
#     plt.plot(timeline,(1-prob_low_risk)/(1-prob_high_risk))
#     plt.plot(timeline,(prob_high_risk)/(prob_low_risk))
#     plt.plot(timeline,(prob_low_risk)/(prob_high_risk))
#     plt.plot(timeline,1-prob_high_risk )
plt.legend(name_keys)
plt.axis([0,5,0,1.05])
plt.ylabel('Survival of 5% at highest risk \n divided by \n survival of 5% at lowest risk')
plt.xlabel('Time in years')

plt.grid(True)
#     plt.show()
# if save_plot: 
#     plt.savefig(folder+plots+plot_folder+'{}_{}_KaplanMeier_highvslow_6080_10p_y{}.png'.format(endpoint,dataset,k),bbox_inches='tight')


In [ ]:
risk_bins